In [1]:
detection_model_path = '/media/prateek/shared_space/useful_models/FIRE/2021_04_26_ssd_inception/openvino_graph/frozen_inference_graph.xml'
device = 'CPU'


In [2]:
import time,cv2,os
import numpy as np
import IPython.display as Disp

In [3]:
os.sys.path.append("../utilities/")
import support_utility_openvino

In [4]:
fire_detection = support_utility_openvino.async_infer(4)
fire_detection.load_model(model_path=detection_model_path,device=device)
output_support = support_utility_openvino.create_plot()

Available Devices :  ['CPU', 'GNA', 'GPU']
OPTIMIZATION_CAPABILITIES for CPU:  ['FP32', 'FP16', 'INT8', 'BIN']
model inputs : dict_keys(['image_tensor'])
model outputs :  dict_keys(['DetectionOutput'])


In [5]:
class_labels = ['fire','smoke','neutral']

In [15]:
channel = '/home/prateek/shared_space/Notebooks/abg/fire/fire-detection-cnn/fire_vid.mp4'
colors = {}
channel =0
cap = cv2.VideoCapture(channel)
cv2.namedWindow("preview",cv2.WINDOW_NORMAL)
ret,frame = cap.read()
# frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
# cap.set(1,2500)
fps = []
thresh = .5
fire_detection.reinit_model()
while True:
    ret,frame = cap.read()
    fps = fps[-100:]
    if not ret:
        break
    t1 = time.time()
    fire_detection.predict(frame)
    if fire_detection.frame_processed>fire_detection.num_requests:
        op_frame,attr, res = fire_detection.postprocess_op()
        op_frame = output_support.label_obj_detection(op_frame,res[0],labels=class_labels,thresh=thresh)

        op_frame = output_support.write_text(op_frame,"FPS = %.3f"%np.mean(fps),text_color = (0,0,0),font_scale=1,font_thickness=2,highlight_color = (127,0,255))
        fps.append(1/(time.time()-t1))

        cv2.imshow("preview",op_frame)
        k = cv2.waitKey(1)
        if k==ord('q'):
            break
cap.release()
cv2.destroyAllWindows()

##### on images

In [23]:
import os
def get_all_file_path(source_dir,ext='.csv'):
    """
    all images with csv extension exist in set of dirs
    """
    op =[]
    for root, dirs, files in os.walk(source_dir):
        for file in files:
            if file.endswith(ext):
                 op.append(os.path.join(os.path.abspath(root), file))
    return op

im_files = get_all_file_path("/media/prateek/shared_space/useful_vids/fire_data/bmcsl/",".jpg\n")
files = np.random.choice(im_files,size = int(1*len(im_files)))

In [25]:
fire_detection.reinit_model()
thresh = .3
fps = []
cv2.namedWindow("preview",cv2.WINDOW_NORMAL)
for f in im_files:
    t1 = time.time()
    frame = cv2.imread(f)
    fire_detection.predict(frame)
    if fire_detection.frame_processed>=fire_detection.num_requests:
        op_frame,attr, res = fire_detection.postprocess_op()
        op_frame = output_support.label_obj_detection(op_frame,res[0],labels=class_labels,thresh=thresh)

        op_frame = output_support.write_text(op_frame,"FPS = %.3f"%np.mean(fps),text_color = (0,0,0),font_scale=1,font_thickness=2,highlight_color = (127,0,255))
        fps.append(1/(time.time()-t1))
        
        cv2.imshow("preview",op_frame)
        k = cv2.waitKey(1)
        if k==ord('q'):
            break
cv2.destroyAllWindows()
    

In [45]:
i = im_files[0]

In [20]:
im_files

[]